In [19]:
from IPython.display import clear_output
!pip install pandas tqdm requests httplib2 google-cloud-language
clear_output()
print('All Installed.')


All Installed.


In [38]:
import httplib2
import pandas as pd
import logging
import time
from tqdm import tqdm
import warnings
import datetime as dt
import os
import string
from datetime import timedelta, date
from urllib.error import HTTPError
from IPython.display import clear_output
import requests
import json

from google.cloud import language
from google.oauth2 import service_account
from google.cloud.language import enums
from google.cloud.language import types

client = language.LanguageServiceClient.from_service_account_json('services.json')

In [39]:
def language_analysis(text):
    content = text
    document = types.Document(
        content=content,
        type=enums.Document.Type.PLAIN_TEXT)
    
    features = {'extract_syntax': True,
                'extract_entities': True,
                'extract_document_sentiment': True,
                'extract_entity_sentiment': True,
                'classify_text': False
                }
    response = client.annotate_text(document=document, features=features)
    sentiment = response.document_sentiment
    entities = response.entities
    
    response = client.classify_text(document)
    categories = response.categories
        
    def get_type(type):
        return client.enums.Entity.Type(entity.type).name
    
    invalid_types = ['OTHER']
    result = {}
    
    result['sentiment'] = []    
    result['entities'] = []
    result['categories'] = []

    if sentiment:
        result['sentiment'] = [{ 'magnitude': sentiment.magnitude, 'score':sentiment.score }]
        
    for entity in entities:
        if get_type(entity.type) not in invalid_types:
            result['entities'].append({'name': entity.name, 'type': get_type(entity.type), 'salience': entity.salience, 'wikipedia_url': entity.metadata.get('wikipedia_url', '-')  })
            
    for category in categories:
        result['categories'].append({'name':category.name, 'confidence': category.confidence})
        
        
    return result

In [40]:
data = pd.read_csv("export_dataframe_cleaner1.csv")
df = data.copy()

In [41]:
df.fillna('', inplace=True)

In [42]:
df["final_text1"] = df["description"] + df["What"] + df["Cite"] + df["Source Description"]
#df["final_text1"] = df["summary_clean"]

In [43]:
c = 0
for i, row in df.iterrows():
    l = len(str(row.final_text1))
    if len(str(row.final_text1).split()) > 20:
        c=c+1
print(c)

241


In [44]:
for i, row in df.iterrows():
    result = ''
    text = ''
    text = str(row.final_text1)
    #print(len(text))
    #print(text)
    if len(text.split()) > 20:
        result = language_analysis(text)
        #print(json.dumps(result, indent=4))
        df.at[i, "Text_Analysis"] = json.dumps(result)

In [18]:
df.to_csv ('text_analysis.csv', index = False, header=True)